## Zadanie domowe: morfologiczna gra w życie – John Conway

### Wykorzystanie operacji LUT w przekształceniu trafi, nie trafi
  - Szybszą metodą wykonania transformacji trafi, nie trafi może być operacja LUT.
  - Technika polega na zakodowaniu wyników wszystkich interesujących  konfiguracji, a następnie podczas przetwarzania wykorzystania operacji LUT.
  - Dla otoczenia 3x3 możliwe jest 512 różnych konfiguracji.
  - Aby praktycznie zrealizować operacje, każdej konfiguracji należy nadać unikalny indeks. Jedną z możliwości jest pomnożenie elementu strukturalnego przez macierz (mnożenie odpowiednich elementów):
  ```
  [[1, 8,  64],
   [ 2, 16, 128],
   [ 4, 32, 256]]
  ```
  Przykładowo elementowi:
  ```
  [[1, 1, 0],
   [ 1, 0, 1],
   [ 1, 0, 1]]
  ```
  odpowiada liczba: 1(1) + 2(1) + 4(1) + 8(1) + 128(1) + 256(1) = 399.
  
### Przykład działania metody – detekcja punktów końcowych na obrazie.
  - założenie: punkt końcowy to punkt, który ma dokładnie jednego sąsiada,
  - zdefiniuj funkcję, która jako argument pobiera otoczenie, a zwraca 0 lub 1 w zależności od tego, czy rozpatrywany punkt jest końcowy np. dla sąsiedztwa 3×3 punkt będzie końcowy, jeżeli jest zapalony i ma tylko jednego sąsiada (czyli suma pikseli jest równa 2).
  - wygeneruj przekodowanie LUT.
  - wczytaj obraz szkielet.bmp (należy go przekształcić, aby uzyskać dwuwymiarową tablicę o wartościach 0-1). Wykorzystując wygenerowane przekodowanie LUT wykonaj detekcję zakończeń. Wyświetl obraz oryginalny i po przekodowaniu LUT.

### Gra w życie

Reguły gry w życie:
  - każdy piksel biały, który ma dwóch lub trzech sąsiadów (białych) przeżywa,
  - każdy piksel biały, który ma 0,1 lub więcej niż trzech sąsiadów (białych) nie przeżywa (głód lub przeludnienie),
  - jeżeli jakieś pole (czarne) sąsiaduje dokładnie z trzema pikselami białymi, to na tym polu ,,rodzi'' się nowy piksel biały.

Zadanie:
  - za pomocą mechanizmu LUT (opisanego wcześniej) należy zaimplementować morfologiczną gre w życie,
  - najważniejszym elementem jest funkcja opisująca reguły gry,
  - symulacje należny przeprowadzić dla plansz dostarczonych w pliku gra.py,
  - dobrze jest wykonać kilka iteracji – zobaczyć jak zmienia się kształt,
  - inne ciekawe kształty do znalezienia w internecie.


In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os

szkielet = cv2.imread('szkielet.bmp',cv2.IMREAD_GRAYSCALE)
szkielet_bool = szkielet.astype(bool)

lut = np.zeros(512)
for i in range(9):
    pos = 16+2**i
    if pos == 32:
        continue
    lut[pos] = 1

el = np.array([[1,8,64],[2,16,128],[4,32,256]])
size = 3
res = np.zeros(szkielet.shape)

for i in range(size//2,szkielet.shape[0]-size//2):
    for j in range(size//2,szkielet.shape[1]-size//2):
        surr = szkielet_bool[i-size//2:i+size//2+1,j-size//2:j+size//2+1]
        res[i,j] = lut[np.sum(np.multiply(surr,el))]

plt.figure()
plt.subplot(1,2,1)
plt.imshow(szkielet,'gray')
plt.axis('off')
plt.subplot(1,2,2)
plt.imshow(res,'gray')
plt.axis('off')
plt.show()

In [ ]:
if not os.path.exists("gra.py") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/10_Morphology/gra.py --no-check-certificate

import gra
p1 = gra.plansza1
p2 = gra.plansza2

el = np.array([[1,8,64],[2,16,128],[4,32,256]])
size = 3
lut = np.zeros(512)
for i in range(512):
    binary = bin(i)[2:]
    if len(binary) >= 5 and binary[len(binary)-5] == '1' and (binary.count('1') == 3 or binary.count('1') == 4):
        lut[i] = 1
    if (len(binary) < 5 and binary.count('1') == 3) or (len(binary) >= 5 and binary[len(binary)-5] == '0' and binary.count('1') == 3):
        lut[i] = 1

def play_game(plansza,iteracje=1):
    plt.figure()
    plt.imshow(plansza*255,'gray')
    plt.title('Plansza początkowa')
    plt.axis('off')
    plt.show()
    for k in range(iteracje):
        padded_plansza = np.pad(plansza, ((1, 1), (1, 1)), mode='constant')
        res = np.zeros(padded_plansza.shape)
        for i in range(size//2,padded_plansza.shape[0]-size//2):
            for j in range(size//2,padded_plansza.shape[1]-size//2):
                surr = padded_plansza[i-size//2:i+size//2+1,j-size//2:j+size//2+1]
                res[i,j] = lut[int(np.sum(np.multiply(surr,el)))]
        res = res[1:res.shape[0]-1,1:res.shape[1]-1]
        plt.figure()
        plt.imshow(res*255,'gray')
        plt.title('Iteracja {}'.format(k+1))
        plt.axis('off')
        plt.show()
        plansza = res

play_game(p1,3)
play_game(p2,3)